In [1]:
import duckdb
import pandas as pd

conn = duckdb.connect()

In [2]:
books_ids = pd.read_csv('data/googlereads/book_id_map.csv')

In [3]:
users_ids = pd.read_csv('data/googlereads/user_id_map.csv')

In [33]:
users_ids

,user_id_csv,user_id
0,0,8842281e1d1347389f2ab93d60773d4d
1,1,72fb0d0087d28c832f15776b0d936598
2,2,ab2923b738ea3082f5f3efcbbfacb218
3,3,d986f354a045ffb91234e4af4d1b12fd
4,4,7504b2aee1ecb5b2872d3da381c6c91e
...,...,...
876140,876140,d890e8079d8896e0cc6c4f178634850a
876141,876141,71ca1aedba3bb07e9e914d072add7e48
876142,876142,d0f6d1a4edcab80a6010cfcfeda4999f
876143,876143,b0883ebf8e31731f1c5d91e678c26666


In [4]:
interactions = pd.read_csv('data/googlereads/goodreads_interactions.csv')

In [28]:
user_group = conn.execute("""
SELECT
  u.user_id_csv,
  u.user_id,
  COUNT(*) as number_of_interactions            
FROM interactions i
  LEFT JOIN users_ids u ON i.user_id = u.user_id_csv
GROUP BY user_id_csv, u.user_id
HAVING COUNT(*) >= 20
ORDER BY 3 DESC
"""
).fetchdf()

In [34]:
user_group

,user_id_csv,user_id,number_of_interactions
0,165733,acb597b9ea2c89354bb8d70d8d4da103,119777
1,339532,2cfc7e879e9a0345ff4f5dec83847566,101295
2,59545,806c0fdc5bed9757ed4d3ca8e3a13be5,82114
3,311008,d3c291b55092951911291f486a4853a6,79168
4,396456,3f238f414bb745aa81d68c3dee14fdc2,77057
...,...,...,...
723203,849352,b5b03a9e5e52fea683fa0a6bd8aaf448,20
723204,849533,eec7e7af519fbb8918df43ec14341d14,20
723205,850869,92df644da6d672c428ac2e7322c46b6b,20
723206,868325,21d44c778cb7ea4db7f3e66a41e67ce7,20


In [ ]:
df_google_reads = conn.execute("""
SELECT
  g.user_id,
  b.book_id,
  i.is_read,
  i.is_reviewed,
  i.rating
FROM interactions i
  LEFT JOIN users_ids g ON i.user_id = g.user_id_csv
  LEFT JOIN books_ids b ON i.book_id = b.book_id_csv
"""
).fetchdf()

In [23]:
df_google_reads

,user_id,book_id,rating
0,9c68bccefde244d300dbad7fe5fcf16f,3,5
1,9c68bccefde244d300dbad7fe5fcf16f,5,5
2,9c68bccefde244d300dbad7fe5fcf16f,136251,5
3,9c68bccefde244d300dbad7fe5fcf16f,6,5
4,9c68bccefde244d300dbad7fe5fcf16f,1,5
...,...,...,...
122410996,310ab2d9b946f53e65d39ebb8f4ae7f5,9434163,0
122410997,310ab2d9b946f53e65d39ebb8f4ae7f5,3922096,0
122410998,310ab2d9b946f53e65d39ebb8f4ae7f5,15507958,4
122410999,310ab2d9b946f53e65d39ebb8f4ae7f5,1374595,0


In [ ]:
df_google_reads.to_parquet(
    path='./data/googlereads/refined/google_reads.parquet.snappy',
    compression='snappy',
    index=False,
)